# <center>1. Знакомимся с данными</center>

✍ Вы уже умеете делать запросы к одной таблице, использовать агрегатные функции и фильтровать данные в выводе. Но в реальных условиях базы данных обычно содержат множество таблиц и при запросе необходимо обращаться к нескольким таблицам. Освоением этого навыка мы сейчас и займёмся ↓

→ Первым делом — знакомимся с датасетом.

В этом модуле мы будем работать с таблицами о футбольных матчах и командах.

Таблицы этого модуля, как и все другие в курсе, лежат в схеме sql в [Metabase](http://sql.skillfactory.ru:3000/). Нам понадобятся таблицы teams и matches.

**Таблица teams с данными о командах**

|Название столбца|Содержимое столбца|
|---|---|
|id	|id команды|
|api_id	|ключ на таблицу matches|
|long_name	|полное название команды|
|short_name	|сокращённое название команды|

**Таблица matches с данными о матчах**

|Название столбца|Содержимое столбца|
|---|---|
|id	|id матча|
|season	|сезон|
|date	|дата матча|
|home_team_api_id	|api_id домашней команды, ключ на таблицу teams|
|away_team_api_id	|api_id гостевой команды, ключ на таблицу teams|
|home_team_goals	|количество голов домашней команды|
|away_team_goals	|количество голов гостевой команды|

Примечание. Как и в предыдущих модулях, вы можете посмотреть подробный разбор структуры предлагаемых запросов во вкладке Детализация.

Исследуйте данные датасета самостоятельно.  
→ Сколько различных полных названий команд в таблице teams?

```sql
SELECT COUNT(DISTINCT long_name) FROM sql.teams
```

→ Сколько в таблице teams команд с коротким названием VAL?

```sql
SELECT COUNT(short_name) FROM sql.teams WHERE short_name='VAL'
```

Информацию о скольких матчах содержит таблица matches?

```sql
SELECT COUNT(*) FROM sql.matches
```

→ Данные за какие сезоны даны в таблице matches?

```sql
SELECT season,COUNT(*) FROM sql.matches GROUP BY season ORDER BY season
```

Напишите запрос, который выведет сезон (season), а также общее количество забитых мячей домашними (total_home_goals) и гостевыми (total_away_goals) командами.  
Отсортируйте по столбцу с сезоном в порядке возрастания.

```sql
SELECT season, SUM(home_team_goals) AS total_home_goals, SUM(away_team_goals) AS total_away_goals
FROM sql.matches
GROUP BY season
ORDER BY season
```

# <center>2. Соединение таблиц по ключу</center>

## <center>Объединяем таблицы без операторов</center>

✍ Существует несколько способов соединения таблиц. Мы познакомимся со всеми основными операторами, которые используются для этих нужд, но начнём с простого метода объединения таблиц — без операторов.

Чтобы соединить две таблицы между собой, достаточно записать названия таблиц через запятую в разделе from. Что произойдёт в таком случае?

```sql
SELECT * /*выбор всех полей*/
FROM
    sql.teams, /*таблица с командами*/
    sql.matches /*таблица с матчами*/
```

Каждая запись, которая есть в таблице teams, будет соединена с каждой записью в таблице matches.  
Это действие также называют декартовым произведением таблиц.

![](data/dst3-u2-md3_2_1.png)

Действительно ли это произведение?

Легко проверить! В исходных таблицах teams и matches было 299 и 25083 записей соответственно. Если соединить каждую запись одной таблицы с каждой записью другой, получится 299 * 25083 записей в итоговой таблице.

Напишите запрос, который выведет количество строк соединённой таблицы.

```sql
SELECT COUNT(*)
FROM sql.teams, sql.matches
```

В данном случае соединение таблиц не даёт практической пользы: мы получили очень много записей, которые никак не можем интерпретировать, потому что команды не соответствуют матчам.

Давайте исправим это. В таблице teams есть столбец api_id, а таблица matches содержит столбцы home_team_api_id и away_team_api_id — это ключи таблиц, по которым они соединяются.

Ключ — это поле (столбец) в таблице, которое позволяет однозначно идентифицировать запись (строку).

Чтобы соединить таблицы и получить данные о домашней команде по каждому матчу, добавим условие
`where home_team_api_id = api_id`.

```sql
SELECT * /*выбор всех полей в таблице*/
FROM
    sql.teams, /*таблица с командами*/
    sql.matches /*таблица с матчами*/
WHERE home_team_api_id = api_id /*условие: home_team_api_id таблицы matches равен api_id таблицы teams*/
```

Аналогично можем получить данные о гостевых командах: необходимо изменить условие на
`where away_team_api_id = api_id`.

```sql
SELECT * /*выбор всех полей в таблицы*/
FROM
    sql.teams, /*таблица с командами*/
    sql.matches /*таблица с матчами*/
WHERE away_team_api_id = api_id /*условие: away_team_api_id таблицы matches равен api_id таблицы teams*/
```

Итак, мы только что объединили таблицы по ключу.

Вы уже знакомы с ключами по таблице pokemon (там в этой роли выступал столбец id). Ключи нужны для того, чтобы иметь возможность не перепутать между собой различные записи.

→ Например, у нас есть несколько команд с одинаковым названием: Polonia Bytom, Widzew Łódź и Royal Excel Mouscron — хотя это разные команды, с разными id.
Кроме того, как мы уже смогли убедиться, ключи используются для соединения таблиц между собой.

Ключи бывают двух основных типов:

1. **Primary** — первичный ключ — служит для идентификации текущей таблицы и, как правило, идёт первым в списке столбцов. Всегда уникален: повторяющихся значений в основной таблице быть не может.
2. **Foreign** — внешний ключ — представляет собой ссылку на другую таблицу.

Как правило, названия ключей имеют «хвост», который позволяет их идентифицировать: например, _id, _rk, _cd, _pk (от primary_key), _fk (от foreign_key) и другие.  
Обратите внимание! В данном датасете ключ api_id таблицы teams может быть использован в разных значениях. Его можно использовать для того, чтобы получить информацию о домашней (home) или гостевой (away) команде.  
Вы могли заметить, что в последних двух запросах получилось очень много столбцов. Как и при работе с одиночной таблицей, мы можем выбирать, какие столбцы соединённой таблицы выводить.  
С помощью известного нам запроса получим названия команд, игравших домашние матчи, и счёт матчей.

```sql
SELECT 
    long_name, /*столбец long_name таблицы teams*/
    home_team_goals, /*столбец home_team_goals таблицы matches*/
    away_team_goals /*столбец away_team_goals таблицы matches*/
FROM
    sql.teams, /*таблица с командами*/
    sql.matches /*таблица с матчами*/
WHERE home_team_api_id = api_id /*условие: home_team_api_id таблицы matches равен api_id таблицы teams*/
```

Напишите запрос, который выведет таблицу с результатами матчей для гостевых команд, содержащую:

* названия гостевых команд (long_name),
* количество забитых мячей домашней команды (home_team_goals),
* количество забитых мячей гостевой команды (away_team_goals).

```sql
SELECT long_name, home_team_goals, away_team_goals
FROM sql.teams, sql.matches
WHERE away_team_api_id = api_id
```

# <center>3. Знакомимся с JOIN</center>

✍ В прошлом юните для соединения таблиц мы использовали условие в разделе where, чтобы показать принцип работы оператора JOIN.

JOIN — это оператор SQL, который позволяет соединять таблицы по условию.

В качестве примера используем запрос из предыдущего юнита.

```sql
SELECT 
    long_name,  /*столбец long_name таблицы teams*/
    home_team_goals, /*столбец home_team_goals таблицы matches*/
    away_team_goals /*столбец away_team_goals таблицы matches*/
FROM
    sql.teams, /*таблица с командами*/
    sql.matches /*таблица с матчами*/
WHERE home_team_api_id = api_id /*условие: home_team_api_id таблицы matches равен api_id таблицы teams*/
```

и запишем его с использованием JOIN.

```sql
SELECT 
    long_name, /*столбец long_name таблицы teams*/
    home_team_goals, /*столбец home_team_goals таблицы matches*/
    away_team_goals /*столбец away_team_goals таблицы matches*/
FROM    
    sql.teams /*таблица с командами*/
JOIN sql.matches on home_team_api_id = api_id /*оператор соединения таблиц; таблица matches; условие: home_team_api_id таблицы matches равен api_id таблицы teams*/
```

Выполните данный запрос

```sql
SELECT *
FROM
    sql.teams,
    sql.matches
WHERE away_team_api_id = api_id

с использованием оператора JOIN, исключив оператор WHERE.

```

```sql
SELECT *
FROM
    sql.teams    
JOIN sql.matches ON away_team_api_id = api_id
```

## <center>Синтаксис</center>

Оператор JOIN упрощает процесс соединения таблиц.

Его синтаксис можно представить следующим образом:

```sql
SELECT
        столбец1,
	столбец2,
	...
FROM
	таблица1
JOIN таблица2 ON условие
```

Порядок присоединения таблиц в данном случае не важен — результат будет одинаковым.

С помощью JOIN можно соединить и более двух таблиц.

```sql
SELECT
        столбец1,
	столбец2,
	...
FROM
	таблица1
JOIN таблица2 ON условие
JOIN таблица3 ON условие
```

В таблицах, которые мы соединяем, могут быть одинаковые названия столбцов.

→ К примеру, столбец id есть и в таблице matches, и в таблице teams. Такой запрос не будет обработан.

```sql
SELECT
    id /*столбец id, но не указано какой таблицы*/
FROM 
sql.teams /*таблица teams*/
    JOIN sql.matches ON home_team_api_id = api_id /*оператор соединения таблиц; таблица matches; условие: home_team_api_id таблицы matches равен api_id таблицы teams*/
```

В результате должно появиться сообщение об ошибке вроде такого: "... column id is ambiguous ...".

? Что же делать в таком случае?

Можно указать, откуда мы хотим запросить данные, записав название таблицы перед столбцом через точку.

```sql
SELECT
    teams.id /*столбец id таблицы teams*/
FROM 
    sql.teams /*таблица teams*/
JOIN sql.matches ON home_team_api_id = api_id /*оператор соединения таблиц; таблица matches; условие: home_team_api_id таблицы matches равен api_id таблицы teams*/
```

Можно также выбрать и столбец из таблицы matches.

```sql
SELECT
    matches.id /*столбец id таблицы matches*/
FROM 
    sql.teams /*таблица teams*/
JOIN sql.matches ON home_team_api_id = api_id /*оператор соединения таблиц; таблица matches; условие: home_team_api_id таблицы matches равен api_id таблицы teams*/
```

Зачастую названия таблиц слишком длинные, так что использовать их неудобно.

Упростить обращение к различным таблицам можно, присвоив им сокращённые названия — алиасы (от англ. alias).

Синтаксис для указания алиаса такой же, как и для названия столбца.

```sql
SELECT
        столбец1,
	столбец2,
	...
FROM
	таблица1 AS короткое_название_1
JOIN таблица2 AS короткое_название_2 ON условие
```

Название записывается без пробелов и операторов.

Если необходимо записать название, в котором используются пробелы ("table 1"), то алиас можно обернуть в кавычки.

Важно! Обращаться по такому алиасу придётся также с помощью кавычек.

```sql
SELECT
	"table 1".столбец1,
	"table 2".столбец2,
	...
FROM
	таблица1 AS "table 1"
	JOIN таблица2 AS "table 2" ON условие
```

Использование таких алиасов считается плохой практикой как минимум по причине того, что обращаться с такими алиасами неудобно.

Вместо алиасов с кавычками рекомендуется давать простое короткое название на латинице, без специальных символов и пробелов. Если таблиц немного и все названия начинаются с разных букв, можно присваивать алиасы по первой букве.

Ключевое слово as, как и в названии столбца, можно опустить в большинстве СУБД.

```sql
SELECT
        столбец1 новое_название_столбца,
	столбец2 новое_название_столбца,
	...
FROM
	таблица1 короткое_название_1
JOIN таблица2 короткое_название_2 ON условие
```

Давайте попробуем проделать то же самое для наших таблиц.

Напишите запрос, который выведет два столбца: id матча (match_id) и id домашней команды (team_id). Отсортируйте по id матча в порядке возрастания значений.

```sql
SELECT
    m.id match_id,
    t.id team_id
FROM
    sql.matches m
    join sql.teams t on t.api_id = m.home_team_api_id
ORDER BY 1
```

Кажется, теперь мы можем соединить таблицы между собой и создать своё табло со счётом матчей, как на спортивных сайтах!  
Давайте с помощью запроса SQL получим таблицу, содержащую:  
* название домашней команды;
* количество забитых домашней командой голов;
* количество забитых гостевой командой голов;
* название гостевой команды.

```sql
SELECT
    h.long_name "домашняя команда", /*столбец long_name таблицы h*/
    m.home_team_goals "голы домашней команды", /*столбец home_team_goals таблицы m*/
    m.away_team_goals "голы гостевой команды", /*столбец away_team_goals таблицы m*/
    a.long_name "гостевая команда" /*столбец long_name таблицы a*/
FROM
    sql.matches m /*таблица matches с алиасом m*/
    JOIN sql.teams h ON m.home_team_api_id = h.api_id /*оператор соединения таблиц; таблица teams с алиасом h; условие: home_team_api_id таблицы m равен api_id таблицы h*/
    JOIN sql.teams a ON m.away_team_api_id = a.api_id /*оператор соединения таблиц; таблица teams с алиасом a; условие: away_team_api_id таблицы m равен api_id таблицы a*/
```

Напишите запрос, который выведет столбцы:

id матча,
короткое название домашней команды (home_short),
короткое название гостевой команды (away_short).
Отсортируйте запрос по возрастанию id матча.

```sql
SELECT 
    m.id,
    t.short_name home_short,
    t1.short_name away_short
FROM sql.matches m
JOIN sql.teams t on t.api_id = m.home_team_api_id
JOIN sql.teams t1 on t1.api_id = m.away_team_api_id 
ORDER BY 1

```

# <center>4. Фильтрация и агрегатные функции</center>

## <center>Работа с объединёнными таблицами</center>

✍ Соединять таблицы мы научились, теперь давайте научимся получать необходимые данные из нескольких таблиц ↓